<a href="https://www.kaggle.com/code/klyushnik/roc-auc0-85153?scriptVersionId=262540514" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/advanced-dls-spring-2021/submission.csv
/kaggle/input/advanced-dls-spring-2021/train.csv
/kaggle/input/advanced-dls-spring-2021/test.csv


In [2]:
import os
import warnings
import logging

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import shap

from scipy import stats
from scipy.optimize import minimize
from scipy.stats import mstats

import catboost
from catboost import CatBoostClassifier
from catboost.utils import get_fnr_curve, get_fpr_curve, get_roc_curve

import lightgbm as lgb
import xgboost as xgb

from mlxtend.classifier import StackingCVClassifier

from sklearn.ensemble import (AdaBoostClassifier, BaggingClassifier,
                              RandomForestClassifier, VotingClassifier)
from sklearn.feature_selection import (SelectKBest, RFECV, chi2,
                                       VarianceThreshold, SequentialFeatureSelector)
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (accuracy_score, classification_report,
                             log_loss, roc_curve, roc_auc_score)
from sklearn.model_selection import (KFold, RepeatedKFold,
                                     RepeatedStratifiedKFold,
                                     StratifiedGroupKFold, StratifiedKFold,
                                     train_test_split)
from sklearn.preprocessing import (LabelEncoder, QuantileTransformer, StandardScaler,
                                   PowerTransformer, MaxAbsScaler, MinMaxScaler,
                                   RobustScaler, PolynomialFeatures, OrdinalEncoder,
                                   OneHotEncoder, FunctionTransformer, KBinsDiscretizer)
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin

import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.holtwinters import ExponentialSmoothing

from category_encoders import TargetEncoder, MEstimateEncoder
# from cuml.preprocessing import TargetEncoder

# from imblearn.over_sampling import (SMOTE, ADASYN,
#                                     BorderlineSMOTE, RandomOverSampler,
#                                     KMeansSMOTE)
# from imblearn.under_sampling import RandomUnderSampler
# from imblearn.pipeline import make_pipeline, Pipeline

import optuna

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras import layers
from tensorflow.keras.initializers import Constant
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
from tensorflow.keras.metrics import AUC
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow import keras

mpl.rcParams.update(mpl.rcParamsDefault)
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
warnings.filterwarnings('ignore')

sns.set_context("notebook", font_scale=1.2)
sns.set_style("whitegrid")

%matplotlib inline

2025-09-18 06:45:12.183840: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1758177912.211726     195 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1758177912.219993     195 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
def plot_numerical_features(df):
    num_features = df.select_dtypes(include=[np.number]).columns
    ncols = 2
    nrows = (len(num_features) + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 6 * nrows))
    axes = axes.flatten()

    for i, feature in enumerate(num_features):
        sns.histplot(df[feature], bins=30, kde=True, ax=axes[i], color='skyblue', edgecolor='black')
        axes[i].set_title(f'Distribution of {feature}', fontsize=18, fontweight='bold')
        axes[i].set_xlabel(feature, fontsize=14)
        axes[i].set_ylabel('Frequency', fontsize=14)
        axes[i].grid(True, linestyle='--', alpha=0.7)  

        mean_value = df[feature].mean()
        axes[i].axvline(mean_value, color='red', linestyle='--', label='Mean')
        axes[i].legend()

    plt.tight_layout()
    plt.show()

def plot_numerical_boxplots(df):
    num_features = df.select_dtypes(include=[np.number]).columns
    ncols = 2
    nrows = (len(num_features) + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 6 * nrows))
    axes = axes.flatten()

    for i, feature in enumerate(num_features):
        sns.boxplot(x=df[feature], ax=axes[i], color='lightgreen')
        axes[i].set_title(f'Boxplot of {feature}', fontsize=18, fontweight='bold')
        axes[i].set_xlabel(feature, fontsize=14)
        axes[i].grid(True, linestyle='--', alpha=0.7)  

        median_value = df[feature].median()
        axes[i].axvline(median_value, color='orange', linestyle='--', label='Median')
        axes[i].legend()

    plt.tight_layout()
    plt.show()

def plot_qq_plot(df):
    num_features = df.select_dtypes(include=[np.number]).columns
    ncols = 2
    nrows = (len(num_features) + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 6 * nrows))
    axes = axes.flatten()

    for i, feature in enumerate(num_features):
        stats.probplot(df[feature], dist="norm", plot=axes[i])
        axes[i].set_title(f'QQ Plot of {feature}', fontsize=18, fontweight='bold')
        axes[i].set_xlabel('Theoretical Quantiles', fontsize=14)
        axes[i].set_ylabel('Sample Quantiles', fontsize=14)
        axes[i].grid(True, linestyle='--', alpha= 0.7)  

    plt.tight_layout()
    plt.show()

def plot_correlation_matrix(df, method='spearman'):
    num_df = df.select_dtypes(include=[np.number])
    
    corr = num_df.corr(method=method)
    plt.figure(figsize=(14, 10))
    sns.heatmap(corr, annot=True, fmt=".2f", cmap='coolwarm', square=True, cbar_kws={"shrink": .8}, linewidths=.5)
    plt.title(f'Correlation Matrix ({method.capitalize()} Correlation)', fontsize=18, fontweight='bold')
    plt.xticks(fontsize=12)
    plt.yticks(fontsize=12)
    plt.show()

def plot_pairplot(df):
    num_features = df.select_dtypes(include=[np.number]).columns
    sns.pairplot(df[num_features], diag_kind='kde', plot_kws={'alpha': 0.6, 'edgecolor': 'k'}, height=2.5)
    plt.suptitle('Pairplot of Numerical Features', y=1.02, fontsize=18, fontweight='bold')
    plt.show()

def plot_categorical_features(df, ncols=2, top_n=None):
    cat_features = df.select_dtypes(include=[object]).columns
    nrows = (len(cat_features) + ncols - 1) // ncols

    fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(14, 6 * nrows))
    axes = axes.flatten()

    for i, feature in enumerate(cat_features):
        if top_n is not None:
            top_categories = df[feature].value_counts().nlargest(top_n).index
            sns.countplot(data=df[df[feature].isin(top_categories)], y=feature, ax=axes[i], palette='viridis', order=top_categories)
        else:
            sns.countplot(data=df, y=feature, ax=axes[i], palette='viridis')
        
        axes[i].set_title(f'Count of {feature}', fontsize=18, fontweight='bold')
        axes[i].set_xlabel('Count', fontsize=14)
        axes[i].set_ylabel(feature, fontsize=14)
        axes[i].tick_params(axis='y', rotation=0)
        axes[i].grid(True, linestyle='--', alpha=0.7)  
    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])

    plt.tight_layout()
    plt.show()

def PolynomialFeatures_labeled(input_df,power):
   
    poly = preprocessing.PolynomialFeatures(power)
    output_nparray = poly.fit_transform(input_df)
    powers_nparray = poly.powers_

    input_feature_names = list(input_df.columns)
    target_feature_names = ["Constant Term"]
    for feature_distillation in powers_nparray[1:]:
        intermediary_label = ""
        final_label = ""
        for i in range(len(input_feature_names)):
            if feature_distillation[i] == 0:
                continue
            else:
                variable = input_feature_names[i]
                power = feature_distillation[i]
                intermediary_label = "%s+%d" % (variable,power)
                if final_label == "":         #If the final label isn't yet specified
                    final_label = intermediary_label
                else:
                    final_label = final_label + "x" + intermediary_label
        target_feature_names.append(final_label)
    output_df = pd.DataFrame(output_nparray, columns = target_feature_names)
    return output_df

def variance_threshold(df,th):
    var_thres=VarianceThreshold(threshold=th)
    var_thres.fit(df)
    new_cols = var_thres.get_support()
    return df.iloc[:,new_cols]
   
def optimize_memory_usage(df, print_size=True):
    """
    Optimizes memory usage in a DataFrame by downcasting numeric columns.

    Parameters:
        df (pd.DataFrame): The DataFrame to optimize.
        print_size (bool): If True, prints memory usage before and after optimization.

    Returns:
        pd.DataFrame: The optimized DataFrame.
    """
    # Types for optimization.
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    
    # Memory usage size before optimize (Mb).
    before_size = df.memory_usage().sum() / 1024**2
    
    for column in df.columns:
        column_type = df[column].dtype
        
        if column_type in numerics:
            try:
                if str(column_type).startswith('int'):
                    df[column] = pd.to_numeric(df[column], downcast='integer')
                else:
                    df[column] = pd.to_numeric(df[column], downcast='float')
                logger.info(f"Optimized column {column}: {column_type} -> {df[column].dtype}")
            except Exception as e:
                logger.error(f"Failed to optimize column {column}: {e}")
    
    # Memory usage size after optimize (Mb).
    after_size = df.memory_usage().sum() / 1024**2
    
    if print_size:
        print(
            'Memory usage size: before {:5.4f} Mb - after {:5.4f} Mb ({:.1f}%).'.format(
                before_size, after_size, 100 * (before_size - after_size) / before_size
            )
        )
    
    return df

In [4]:
train = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/train.csv")
test = pd.read_csv("/kaggle/input/advanced-dls-spring-2021/test.csv")

display(train.shape, test.shape)
display(train.info(), test.info())

# test = test.drop(['id'], axis =1)
# train = train.drop(['id'], axis =1)

display(train.describe().T)
display(test.describe().T)

duplicates = train.duplicated()
print(f"Number of duplicates: {duplicates.sum()}")

duplicates = test.duplicated()
print(f"Number of duplicates: {duplicates.sum()}")

for col in train.columns:
    pct_missing = np.mean(train[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

display(train.head(5))

for col in test.columns:
    pct_missing = np.mean(test[col].isnull())
    print('{} - {}%'.format(col, round(pct_missing*100)))

train = optimize_memory_usage(train)
test = optimize_memory_usage(test)

(5282, 20)

(1761, 19)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5282 entries, 0 to 5281
Data columns (total 20 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   ClientPeriod              5282 non-null   int64  
 1   MonthlySpending           5282 non-null   float64
 2   TotalSpent                5282 non-null   object 
 3   Sex                       5282 non-null   object 
 4   IsSeniorCitizen           5282 non-null   int64  
 5   HasPartner                5282 non-null   object 
 6   HasChild                  5282 non-null   object 
 7   HasPhoneService           5282 non-null   object 
 8   HasMultiplePhoneNumbers   5282 non-null   object 
 9   HasInternetService        5282 non-null   object 
 10  HasOnlineSecurityService  5282 non-null   object 
 11  HasOnlineBackup           5282 non-null   object 
 12  HasDeviceProtection       5282 non-null   object 
 13  HasTechSupportAccess      5282 non-null   object 
 14  HasOnlin

None

None

,count,mean,std,min,25%,50%,75%,max
ClientPeriod,5282.0,32.397009,24.550326,0.00,9.0000,29.0,55.00,72.00
MonthlySpending,5282.0,64.924754,30.176464,18.25,35.4625,70.4,90.05,118.75
IsSeniorCitizen,5282.0,0.159409,0.366092,0.00,0.0000,0.0,0.00,1.00
Churn,5282.0,0.262022,0.439776,0.00,0.0000,0.0,1.00,1.00


,count,mean,std,min,25%,50%,75%,max
ClientPeriod,1761.0,32.293583,24.593736,0.00,9.00,29.00,56.0,72.00
MonthlySpending,1761.0,64.272601,29.832517,18.55,35.65,70.05,89.4,117.35
IsSeniorCitizen,1761.0,0.170358,0.376054,0.00,0.00,0.00,0.0,1.00


Number of duplicates: 14
Number of duplicates: 3
ClientPeriod - 0%
MonthlySpending - 0%
TotalSpent - 0%
Sex - 0%
IsSeniorCitizen - 0%
HasPartner - 0%
HasChild - 0%
HasPhoneService - 0%
HasMultiplePhoneNumbers - 0%
HasInternetService - 0%
HasOnlineSecurityService - 0%
HasOnlineBackup - 0%
HasDeviceProtection - 0%
HasTechSupportAccess - 0%
HasOnlineTV - 0%
HasMovieSubscription - 0%
HasContractPhone - 0%
IsBillingPaperless - 0%
PaymentMethod - 0%
Churn - 0%


,ClientPeriod,MonthlySpending,TotalSpent,Sex,IsSeniorCitizen,HasPartner,HasChild,HasPhoneService,HasMultiplePhoneNumbers,HasInternetService,HasOnlineSecurityService,HasOnlineBackup,HasDeviceProtection,HasTechSupportAccess,HasOnlineTV,HasMovieSubscription,HasContractPhone,IsBillingPaperless,PaymentMethod,Churn
0,55,19.50,1026.35,Male,0,Yes,Yes,Yes,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,One year,No,Mailed check,0
1,72,25.85,1872.2,Male,0,Yes,No,Yes,Yes,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Two year,No,Credit card (automatic),0
2,1,75.90,75.9,Male,0,No,No,Yes,No,Fiber optic,No,No,No,Yes,No,No,Month-to-month,Yes,Electronic check,1
3,32,79.30,2570,Female,1,Yes,No,Yes,Yes,Fiber optic,No,No,Yes,No,No,No,Month-to-month,No,Mailed check,0
4,60,115.25,6758.45,Female,0,Yes,Yes,Yes,Yes,Fiber optic,Yes,Yes,Yes,Yes,Yes,Yes,Two year,No,Credit card (automatic),0


ClientPeriod - 0%
MonthlySpending - 0%
TotalSpent - 0%
Sex - 0%
IsSeniorCitizen - 0%
HasPartner - 0%
HasChild - 0%
HasPhoneService - 0%
HasMultiplePhoneNumbers - 0%
HasInternetService - 0%
HasOnlineSecurityService - 0%
HasOnlineBackup - 0%
HasDeviceProtection - 0%
HasTechSupportAccess - 0%
HasOnlineTV - 0%
HasMovieSubscription - 0%
HasContractPhone - 0%
IsBillingPaperless - 0%
PaymentMethod - 0%
Memory usage size: before 0.8061 Mb - after 0.6802 Mb (15.6%).
Memory usage size: before 0.2554 Mb - after 0.2252 Mb (11.8%).


In [5]:
class DataPreprocessor:
    def __init__(self):
        self.scaler = StandardScaler()
        self.is_fitted = False
        
    def fit_transform(self, df, target_present=True):
        """Обучение и преобразование тренировочных данных"""
        df_processed = self._basic_preprocessing(df)
        
        # Масштабирование числовых признаков
        numerical_columns = ['ClientPeriod', 'MonthlySpending', 'TotalSpent', 
                            'SpentPerMonth', 'SpendingRatio']
        
        df_processed[numerical_columns] = self.scaler.fit_transform(df_processed[numerical_columns])
        self.is_fitted = True
        
        if target_present and 'Churn' in df_processed.columns:
            return df_processed.drop('Churn', axis=1), df_processed['Churn']
        else:
            return df_processed
        
    def transform(self, df, target_present=True):
        """Преобразование тестовых данных"""
        if not self.is_fitted:
            raise ValueError("Сначала вызовите fit_transform на тренировочных данных")
            
        df_processed = self._basic_preprocessing(df)
        
        numerical_columns = ['ClientPeriod', 'MonthlySpending', 'TotalSpent', 
                            'SpentPerMonth', 'SpendingRatio']
        
        df_processed[numerical_columns] = self.scaler.transform(df_processed[numerical_columns])
        
        if target_present and 'Churn' in df_processed.columns:
            return df_processed.drop('Churn', axis=1), df_processed['Churn']
        else:
            return df_processed
    
    def _basic_preprocessing(self, df):
        """Базовая предобработка, общая для train и test"""
        df_processed = df.copy()
        
        # Все шаги предобработки кроме масштабирования...
        df_processed['TotalSpent'] = pd.to_numeric(df_processed['TotalSpent'], errors='coerce').fillna(0)
        
        binary_columns = [
            'Sex', 'HasPartner', 'HasChild', 'HasPhoneService', 
            'HasMultiplePhoneNumbers', 'HasInternetService',
            'HasOnlineSecurityService', 'HasOnlineBackup', 
            'HasDeviceProtection', 'HasTechSupportAccess',
            'HasOnlineTV', 'HasMovieSubscription', 
            'HasContractPhone', 'IsBillingPaperless'
        ]
        
        for col in binary_columns:
            df_processed[col] = df_processed[col].str.lower()
            df_processed[col] = df_processed[col].map({
                'yes': 1, 'no': 0, 'male': 1, 'female': 0,
                'one year': 1, 'two year': 1, 'month-to-month': 0,
                'no internet service': 0
            }).fillna(0).astype(int)
        
        payment_method_mapping = {
            'Mailed check': 0, 'Credit card (automatic)': 1,
            'Bank transfer (automatic)': 2, 'Electronic check': 3
        }
        
        df_processed['PaymentMethod'] = df_processed['PaymentMethod'].map(
            payment_method_mapping
        ).fillna(0).astype(int)
        
        # Создание новых признаков
        df_processed['SpentPerMonth'] = df_processed['TotalSpent'] / np.where(
            df_processed['ClientPeriod'] > 0, 
            df_processed['ClientPeriod'], 
            1
        )
        
        df_processed['SpendingRatio'] = df_processed['MonthlySpending'] / df_processed['SpentPerMonth'].replace(0, 1)
        
        return df_processed

# Использование:
preprocessor = DataPreprocessor()

# Обработка тренировочных данных
X, y = preprocessor.fit_transform(train)

# Обработка тестовых данных (без целевой переменной)
test = preprocessor.transform(test, target_present=False)

In [6]:
X = variance_threshold(X,0.03)
list_name = (X.columns)
test = test[list_name]

display(X.shape, y.shape, test.shape)

(5282, 20)

(5282,)

(1761, 20)

In [7]:
def create_ensemble_classification_auc(X, y, test_aligned, n_folds=7):
    """
    Ансамбль моделей для классификации с использованием ROC-AUC score
    
    Parameters:
    X (pd.DataFrame): Признаки train
    y (pd.Series): Целевая переменная train
    test_aligned (pd.DataFrame): Признаки test
    n_folds (int): Количество фолдов для кросс-валидации
    
    Returns:
    oof_df, predictions_df, model_info
    """
    FOLDS = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    all_oof_proba = {}  # OOF вероятности для ROC-AUC
    all_predictions_proba = {}  # Тестовые вероятности
    models = []
    
    # CatBoost модели 
    models.append(('cat_1', CatBoostClassifier(verbose=0, random_state=42, 
                                                auto_class_weights='Balanced',
                                                iterations=500,
                                                learning_rate=0.05,
                                                depth=7,
                                                l2_leaf_reg=5)))
    
    models.append(('cat_2', CatBoostClassifier(verbose=0, random_state=42,
                                                iterations=400,
                                                learning_rate=0.1,
                                                depth=4,
                                                bootstrap_type='Bernoulli',
                                                subsample=0.8)))
    
    models.append(('cat_3', CatBoostClassifier( verbose=0, random_state=42,
                                                iterations=300,
                                                learning_rate=0.03,
                                                depth=10,
                                                l2_leaf_reg=10,
                                                grow_policy='Lossguide')))
    
    models.append(('cat_balanced', CatBoostClassifier(
                                                        verbose=0, random_state=42, 
                                                        auto_class_weights='Balanced',
                                                        iterations=1000,
                                                        learning_rate=0.05,
                                                        depth=7,
                                                        l2_leaf_reg=5
                                                        )))
    
    models.append(('cat_fast', CatBoostClassifier(
                                                    verbose=0, random_state=42,
                                                    iterations=300,
                                                    learning_rate=0.1,
                                                    depth=4,
                                                    bootstrap_type='Bernoulli',
                                                    subsample=0.8
                                                )))
    
    models.append(('cat_deep', CatBoostClassifier(
                                                    verbose=0, random_state=42,
                                                    iterations=800,
                                                    learning_rate=0.03,
                                                    depth=10,
                                                    l2_leaf_reg=10,
                                                    grow_policy='Lossguide'
                                                )))
    
    models.append(('cat_regularized', CatBoostClassifier(
                                                            verbose=0, random_state=42,
                                                            iterations=600,
                                                            learning_rate=0.02,
                                                            depth=6,
                                                            l2_leaf_reg=20,
                                                            border_count=128,
                                                            random_strength=2
                                                        )))
    
    models.append(('cat_high_lr', CatBoostClassifier(
                                                        verbose=0, random_state=42,
                                                        iterations=200,
                                                        learning_rate=0.2,
                                                        depth=5,
                                                    )))
    
    # XGBoost модели
    models.append(('xgb_1', xgb.XGBClassifier(random_state=42, n_jobs=1,
                                                use_label_encoder=False, eval_metric='logloss',
                                                n_estimators=300,
                                                learning_rate=0.1,
                                                max_depth=4,
                                                booster='dart',
                                                rate_drop=0.1)))
    
    models.append(('xgb_2', xgb.XGBClassifier( random_state=42, n_jobs=1,
                                                use_label_encoder=False, eval_metric='logloss',
                                                n_estimators=50,
                                                learning_rate=0.02,
                                                max_depth=7,
                                                reg_alpha=1.0,
                                                reg_lambda=2.0,
                                                subsample=0.7)))
    
    models.append(('xgb_3', xgb.XGBClassifier( random_state=42, n_jobs=1,
                                                use_label_encoder=False, eval_metric='logloss',
                                                n_estimators=250,
                                                learning_rate=0.1,
                                                booster='gblinear',
                                                updater='coord_descent')))

    models.append(('xgb_balanced', xgb.XGBClassifier(
                                                    random_state=42, n_jobs=1,
                                                    use_label_encoder=False, eval_metric='logloss',
                                                    n_estimators=100,
                                                    learning_rate=0.05,
                                                    max_depth=6,
                                                    scale_pos_weight=len(y[y==0])/len(y[y==1]) if sum(y) > 0 else 1,
                                                    subsample=0.8,
                                                    colsample_bytree=0.8
                                                )))
    
    models.append(('xgb_fast', xgb.XGBClassifier(
                                                    random_state=42, n_jobs=1,
                                                    use_label_encoder=False, eval_metric='logloss',
                                                    n_estimators=300,
                                                    learning_rate=0.1,
                                                    max_depth=4,
                                                    booster='dart',
                                                    rate_drop=0.1
                                                )))
    
    models.append(('xgb_deep', xgb.XGBClassifier(
                                                    random_state=42, n_jobs=1,
                                                    use_label_encoder=False, eval_metric='logloss',
                                                    n_estimators=500,
                                                    learning_rate=0.03,
                                                    max_depth=12,
                                                    min_child_weight=3,
                                                    gamma=0.2
                                                )))
    
    models.append(('xgb_regularized', xgb.XGBClassifier(
                                                            random_state=42, n_jobs=1,
                                                            use_label_encoder=False, eval_metric='logloss',
                                                            n_estimators=800,
                                                            learning_rate=0.02,
                                                            max_depth=7,
                                                            reg_alpha=1.0,
                                                            reg_lambda=2.0,
                                                            subsample=0.7
                                                        )))
    
    models.append(('xgb_gblinear', xgb.XGBClassifier(
                                                        random_state=42, n_jobs=1,
                                                        use_label_encoder=False, eval_metric='logloss',
                                                        n_estimators=500,
                                                        learning_rate=0.1,
                                                        booster='gblinear',
                                                        updater='coord_descent'
                                                    )))
    
    # LightGBM модели
    models.append(('lgb_1', lgb.LGBMClassifier( verbose=-1,random_state=42, n_jobs=1,
                                                n_estimators=300,
                                                learning_rate=0.05,
                                                num_leaves=63,
                                                class_weight='balanced',
                                                subsample=0.8,
                                                colsample_bytree=0.8)))
    
    models.append(('lgb_2', lgb.LGBMClassifier(  verbose=-1, random_state=42, n_jobs=1,
                                                    n_estimators=400,
                                                    learning_rate=0.03,
                                                    num_leaves=255,
                                                    max_depth=-1,
                                                    min_data_in_leaf=50,
                                                    feature_fraction=0.7)))
    
    models.append(('lgb_3', lgb.LGBMClassifier( verbose=-1, random_state=42, n_jobs=1,
                                                n_estimators=290,
                                                learning_rate=0.02,
                                                num_leaves=127,
                                                reg_alpha=1.0,
                                                reg_lambda=2.0,
                                                subsample_freq=5)))

    models.append(('lgb_balanced', lgb.LGBMClassifier(
                                                        random_state=42, n_jobs=1,
                                                        n_estimators=100,
                                                        learning_rate=0.05,
                                                        num_leaves=63,
                                                        class_weight='balanced',
                                                        subsample=0.8,
                                                        colsample_bytree=0.8
                                                    )))
    
    models.append(('lgb_fast', lgb.LGBMClassifier(
                                                    random_state=42, n_jobs=1,
                                                    n_estimators=300,
                                                    learning_rate=0.1,
                                                    num_leaves=31,
                                                    boosting_type='dart',
                                                    drop_rate=0.1
                                                )))
    
    models.append(('lgb_large', lgb.LGBMClassifier(
                                                    random_state=42, n_jobs=1,
                                                    n_estimators=500,
                                                    learning_rate=0.03,
                                                    num_leaves=255,
                                                    max_depth=-1,
                                                    min_data_in_leaf=50,
                                                    feature_fraction=0.7
                                                )))
    
    models.append(('lgb_regularized', lgb.LGBMClassifier(
                                                            random_state=42, n_jobs=1,
                                                            n_estimators=800,
                                                            learning_rate=0.02,
                                                            num_leaves=127,
                                                            reg_alpha=1.0,
                                                            reg_lambda=2.0,
                                                            subsample_freq=5
                                                        )))
    
    models.append(('lgb_goss', lgb.LGBMClassifier(
                                                    random_state=42, n_jobs=1,
                                                    n_estimators=600,
                                                    learning_rate=0.04,
                                                    num_leaves=95,
                                                    boosting_type='goss',
                                                    top_rate=0.2,
                                                    other_rate=0.1
                                                )))
    
    models.append(('lgb_simple', lgb.LGBMClassifier(
                                                        random_state=42, n_jobs=1,
                                                        n_estimators=400,
                                                        learning_rate=0.08,
                                                        num_leaves=15,
                                                        max_depth=4,
                                                        min_child_samples=100
                                                    )))

    for name, model in models:
        try:
            print(f"\nTraining {name}...")
            oof_proba = np.zeros(len(X))  # OOF вероятности класса 1
            pred_proba = np.zeros(len(test_aligned))  # Тестовые вероятности класса 1
            
            for fold, (trn_idx, val_idx) in enumerate(FOLDS.split(X, y)):
                X_train, y_train = X.iloc[trn_idx], y.iloc[trn_idx]
                X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
                
                # Обучение модели
                if name.startswith('cat'):
                    model.fit(X_train, y_train, verbose=0)
                else:
                    model.fit(X_train, y_train)
                
                # Предсказания вероятностей для OOF
                if hasattr(model, 'predict_proba'):
                    val_proba = model.predict_proba(X_val)[:, 1]
                    oof_proba[val_idx] = val_proba
                    
                    # Предсказания вероятностей для test
                    test_fold_proba = model.predict_proba(test_aligned)[:, 1]
                    pred_proba += test_fold_proba / FOLDS.n_splits
                
                # Расчет ROC-AUC для фолда
                fold_auc = roc_auc_score(y_val, val_proba)
                print(f'{name} - Fold {fold} ROC-AUC: {fold_auc:.4f}')
            
            # Сохраняем OOF вероятности
            all_oof_proba[name] = oof_proba
            
            # Сохраняем тестовые вероятности
            all_predictions_proba[name] = pred_proba
            
            # Полный ROC-AUC score
            full_auc = roc_auc_score(y, oof_proba)
            print(f'{name} - Full OOF ROC-AUC: {full_auc:.4f}')
            
        except Exception as e:
            print(f"Error training {name}: {str(e)}")
            continue
    
    # Проверка, что хотя бы одна модель обучилась
    if not all_oof_proba:
        print("Все модели завершились с ошибкой! Возвращаем None.")
        return None, None, None
    
    # Создаем DataFrame с результатами
    oof_df = pd.DataFrame(all_oof_proba)
    oof_df = oof_df.add_suffix('_proba')
    
    predictions_df = pd.DataFrame(all_predictions_proba)
    predictions_df = predictions_df.add_suffix('_proba')
    
    # Добавляем целевые переменные
    oof_df['target'] = y.values
    
    # Создаем бинарные предсказания на основе порога 0.5 (опционально)
    for model_name in all_oof_proba.keys():
        oof_df[f'{model_name}_pred'] = (oof_df[f'{model_name}_proba'] >= 0.5).astype(int)
        predictions_df[f'{model_name}_pred'] = (predictions_df[f'{model_name}_proba'] >= 0.5).astype(int)
    
    model_info = {
        'model_names': list(all_oof_proba.keys()),
        'num_models': len(all_oof_proba),
        'features_used': list(X.columns),
        'metric_used': 'roc_auc',
        'oof_auc_scores': {name: roc_auc_score(y, all_oof_proba[name]) for name in all_oof_proba.keys()}
    }
    
    # Вывод среднего ROC-AUC
    avg_auc = np.mean(list(model_info['oof_auc_scores'].values()))
    print(f"\nAverage OOF ROC-AUC across all models: {avg_auc:.4f}")
    
    return oof_df, predictions_df, model_info

# Альтернативная версия с возвратом только вероятностей (без бинарных предсказаний)
def create_ensemble_classification_auc_proba_only(X, y, test_aligned, n_folds=7):
    """
    Упрощенная версия, возвращающая только вероятности
    """
    FOLDS = KFold(n_splits=n_folds, shuffle=True, random_state=42)
    
    all_oof_proba = {}
    all_predictions_proba = {}
    models = []
    
    # Модели (можно настроить под конкретную задачу)
    models.append(('cat', CatBoostClassifier(verbose=0, random_state=42)))
    models.append(('xgb', xgb.XGBClassifier(random_state=42, n_jobs=1, use_label_encoder=False, eval_metric='logloss')))
    models.append(('lgb', lgb.LGBMClassifier(random_state=42, n_jobs=1)))

    for name, model in models:
        try:
            print(f"\nTraining {name}...")
            oof_proba = np.zeros(len(X))
            pred_proba = np.zeros(len(test_aligned))
            
            for fold, (trn_idx, val_idx) in enumerate(FOLDS.split(X, y)):
                X_train, y_train = X.iloc[trn_idx], y.iloc[trn_idx]
                X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
                
                if name.startswith('cat'):
                    model.fit(X_train, y_train, verbose=0)
                else:
                    model.fit(X_train, y_train)
                
                val_proba = model.predict_proba(X_val)[:, 1]
                oof_proba[val_idx] = val_proba
                pred_proba += model.predict_proba(test_aligned)[:, 1] / FOLDS.n_splits
                
                fold_auc = roc_auc_score(y_val, val_proba)
                print(f'{name} - Fold {fold} AUC: {fold_auc:.4f}')
            
            all_oof_proba[name] = oof_proba
            all_predictions_proba[name] = pred_proba
            
            full_auc = roc_auc_score(y, oof_proba)
            print(f'{name} - Full OOF AUC: {full_auc:.4f}')
            
        except Exception as e:
            print(f"Error training {name}: {str(e)}")
            continue
    
    if not all_oof_proba:
        return None, None, None
    
    oof_df = pd.DataFrame(all_oof_proba)
    predictions_df = pd.DataFrame(all_predictions_proba)
    oof_df['target'] = y.values
    
    model_info = {
        'model_names': list(all_oof_proba.keys()),
        'auc_scores': {name: roc_auc_score(y, all_oof_proba[name]) for name in all_oof_proba.keys()}
    }
    
    return oof_df, predictions_df, model_info

In [ ]:
# Запуск ансамбля с ROC-AUC
oof_results, test_predictions, model_info = create_ensemble_classification_auc(
    X, y, test, n_folds=5
)

# Анализ результатов
if oof_results is not None:
    print("\n=== Результаты ансамбля ===")
    for model_name, auc_score in model_info['oof_auc_scores'].items():
        print(f"{model_name}: ROC-AUC = {auc_score:.4f}")
    
    # Создание блендинга (усреднение вероятностей)
    proba_columns = [col for col in test_predictions.columns if col.endswith('_proba')]
    test_predictions['ensemble_proba'] = test_predictions[proba_columns].mean(axis=1)
    test_predictions['ensemble_pred'] = (test_predictions['ensemble_proba'] >= 0.5).astype(int)
    
    print(f"\nEnsemble ROC-AUC: {np.mean(list(model_info['oof_auc_scores'].values())):.4f}")


Training cat_1...
cat_1 - Fold 0 ROC-AUC: 0.8149
cat_1 - Fold 1 ROC-AUC: 0.8229
cat_1 - Fold 2 ROC-AUC: 0.8281
cat_1 - Fold 3 ROC-AUC: 0.8584
cat_1 - Fold 4 ROC-AUC: 0.8237
cat_1 - Full OOF ROC-AUC: 0.8295

Training cat_2...
cat_2 - Fold 0 ROC-AUC: 0.8134
cat_2 - Fold 1 ROC-AUC: 0.8299
cat_2 - Fold 2 ROC-AUC: 0.8328
cat_2 - Fold 3 ROC-AUC: 0.8548
cat_2 - Fold 4 ROC-AUC: 0.8295
cat_2 - Full OOF ROC-AUC: 0.8319

Training cat_3...
cat_3 - Fold 0 ROC-AUC: 0.8208
cat_3 - Fold 1 ROC-AUC: 0.8316
cat_3 - Fold 2 ROC-AUC: 0.8342
cat_3 - Fold 3 ROC-AUC: 0.8672
cat_3 - Fold 4 ROC-AUC: 0.8437
cat_3 - Full OOF ROC-AUC: 0.8392

Training cat_balanced...
cat_balanced - Fold 0 ROC-AUC: 0.8099
cat_balanced - Fold 1 ROC-AUC: 0.8178
cat_balanced - Fold 2 ROC-AUC: 0.8204
cat_balanced - Fold 3 ROC-AUC: 0.8514
cat_balanced - Fold 4 ROC-AUC: 0.8179
cat_balanced - Full OOF ROC-AUC: 0.8234

Training cat_fast...
cat_fast - Fold 0 ROC-AUC: 0.8154
cat_fast - Fold 1 ROC-AUC: 0.8319
cat_fast - Fold 2 ROC-AUC: 0.8362

In [ ]:
sample = pd.read_csv('/kaggle/input/advanced-dls-spring-2021/submission.csv')
sample['Churn'] = test_predictions['ensemble_proba']
sample.to_csv('submission.csv', index=False)
sample.head(10)